In [2]:
import pandas as pd
import numpy as np

In [3]:
filename = './positiveSamples/AP.csv'
#savename = './test/AP'

In [4]:
# TODO: 
# 1. Choose better negative sample (ppl with no complications)
# 2. Try different complications and subset broader
# 3. Subset patients with less than 4 visits
# 4. Preprocess Lab results earlier

In [5]:
# Filtering and processing
# 1. Combine lab and exam sheets
# 2. Lab by 18 desired tests
# 3. Coerce lab & exam values to float
# 4. Remove null for date performed 
# 5. Interquartile ranges
# 6. Datetime

# 7. Disease period range
# 8. Remove redundent visits
# 9. 3 < n < 51


## Modified Start using clean Lab/Exam sheet

In [6]:
#experimental new code
lab = pd.read_csv('FullData/LabExamCleanO10K.csv')
DiseaseCase = pd.read_csv("FullData/DiseaseCase.csv")
# uses different lab tests based on biggest diff. between pos and neg pat sets
tests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
lab = lab.loc[lab.Name_calc.isin(tests)]
lab['Name_calc'].value_counts()

Hemoglobin (Hb)                                        1276642
Hemoglobin A1c (HbA1c)                                 1269975
Platelets in Blood (PLT)                               1000336
Leukocytes in Blood (WBC)                               997770
Erythrocytes in Blood (RBC)                             951110
Plasma Creatinine Clearance (eGFR)                      936623
Total Cholesterol (TCh)                                 826151
BMI (kg/m^2)                                            793064
Alanine Aminotransferase in Serum or Plasma (ALT)       743651
Fasting Blood Glucose (FBG)                             659506
Aspartate Aminotransferase in Serum or Plasma (AST)     258311
Total Bilirubin (TBIL)                                  243388
Gamma Glutamyl Transferase in Serum or Plasma (GGT)     144289
Sodium (mM)                                             122764
Potassium (mM)                                          116961
Urea (mM)                                              

In [7]:
# 6. Datetime
lab['PerformedDate'] = pd.to_datetime(lab.PerformedDate,format='%Y-%m-%d')

In [8]:
disease_periods = pd.read_csv(filename, index_col=None)
disease_periods = disease_periods.drop(disease_periods.columns[0], axis=1)
#if len(disease_periods) > 2000:
 #   disease_periods = disease_periods.iloc[0:2000]
patient_records = lab.loc[lab['Patient_ID'].isin(disease_periods['Patient_ID'])]
patient_records = pd.merge(patient_records, disease_periods, on='Patient_ID')
print(len(patient_records.index),patient_records.columns)
patient_records.head()

315405 Index(['Unnamed: 0', 'Patient_ID', 'PerformedDate', 'Name_calc',
       'TestResult_calc', 'UnitOfMeasure_calc', 'CompOnset', 'DateOfOnset'],
      dtype='object')


,Unnamed: 0,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,8,1000100164719,2013-10-11,Platelets in Blood (PLT),225.0,10^9/L,2019-07-25,2013-03-26
1,9,1000100164719,2013-09-22,Platelets in Blood (PLT),243.0,10^9/L,2019-07-25,2013-03-26
2,17,1000100164719,2016-02-22,Leukocytes in Blood (WBC),6.2,10^9/L,2019-07-25,2013-03-26
3,22,1000100164719,2016-09-12,Hemoglobin A1c (HbA1c),7.1,%,2019-07-25,2013-03-26
4,24,1000100164719,2013-10-11,Erythrocytes in Blood (RBC),5.0,10^12/L,2019-07-25,2013-03-26


In [9]:
relVisits = patient_records

In [10]:
relVisits.CompOnset = pd.to_datetime(relVisits.CompOnset,format='%Y-%m-%d')
relVisits.DateOfOnset = pd.to_datetime(relVisits.DateOfOnset,format='%Y-%m-%d')

In [11]:
relVisits = patient_records[(patient_records['PerformedDate'] < patient_records['CompOnset']) & (patient_records['PerformedDate'] >= patient_records['DateOfOnset'])]
len(relVisits.index)

111683

In [12]:
pos_Results = relVisits

In [13]:
# ## 7. Remove Redundent Visits later
pos_Results = pos_Results.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
pos_Results = pos_Results.reset_index(drop = True)

rem = []
for index, row in pos_Results.iterrows():
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))



685


In [14]:
## NEW
pos_Results_u = pos_Results
print(len(pos_Results_u.index))
pos_Results_u = pos_Results_u.drop(rem)
print(len(pos_Results_u.index))
print(len(pos_Results_u['Patient_ID'].unique()))

111683
110998
2308


In [15]:
pos_r_final = pos_Results_u[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() 
pos_r_final = pos_r_final[(pos_r_final['PerformedDate'] < 51) & (pos_r_final['PerformedDate'] > 3)]
pos_r_final

,PerformedDate
Patient_ID,
1000100001914,38
1000100001950,32
1000100003164,27
1000100003603,8
1000100003646,5
...,...
12120070011568,9
12120070011754,5
12120120003869,8


In [16]:
n = len(pos_r_final.index)
print(n)
posFinal = pos_Results_u[pos_Results_u['Patient_ID'].isin(pos_r_final.index)]
print(len(relVisits.index) / n) # Average results per person
posFinal.head()

1650
67.68666666666667


,Unnamed: 0,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
19,21338004,1000100001914,2006-10-03,BMI (kg/m^2),25.6,kg/m^2,2013-07-16,2006-08-28
20,2154482,1000100001914,2007-01-08,Leukocytes in Blood (WBC),5.7,10^9/L,2013-07-16,2006-08-28
21,2154484,1000100001914,2007-01-08,Hemoglobin (Hb),143.0,g/L,2013-07-16,2006-08-28
22,2154486,1000100001914,2007-01-08,Platelets in Blood (PLT),213.0,10^9/L,2013-07-16,2006-08-28
23,2154530,1000100001914,2007-01-08,Fasting Blood Glucose (FBG),9.5,mmol/L,2013-07-16,2006-08-28


In [17]:
# Disease Period
dp = (posFinal['CompOnset'] - posFinal['DateOfOnset']).mean()
print(type(dp))
print(dp)

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
2243 days 23:14:38.631385344


## Negative Sampling
1. Negative Complement
2. Age and Gender Match
3. Visit Subsetting

In [18]:
# Negative sample part 1
# Import patients with complications
compIDs = pd.read_csv('FullData/compIDs.csv')
compIDs = compIDs['0']

# Negative Complement and demographic matching
from random import sample

PatientInfo = pd.read_csv('FullData/Patient.csv')

all_neg_patients = lab.loc[~lab.Patient_ID.isin(compIDs)].Patient_ID.unique()

pos_patients = posFinal['Patient_ID'].unique()
neg_patients = pd.Series([], dtype = 'int64')
size = len(pos_patients)
#takes 45 sec ish to run
for i in range(size):
    refPatient_ID = pos_patients[i]
    refYOB = PatientInfo.loc[PatientInfo.Patient_ID == refPatient_ID].BirthYear.iloc[0]
    refSex = PatientInfo.loc[PatientInfo.Patient_ID == refPatient_ID].Sex.iloc[0]
    PossiblePatients = PatientInfo.loc[PatientInfo.Patient_ID.isin(all_neg_patients) 
                                       & (PatientInfo.Sex == refSex)]
    selectedPat = False
    yearGap = 0
    while not selectedPat:
        #print(len(PossiblePatients))
        possible = PossiblePatients.loc[(PossiblePatients.BirthYear >= refYOB - yearGap) 
                                        & (PossiblePatients.BirthYear <= refYOB + yearGap)].Patient_ID
        if len(possible > 1):
            samp = possible.sample(2)
            selectedPat = samp.iloc[0]
            selectedPat2 = samp.iloc[1]
            neg_patients = neg_patients.append(pd.Series([selectedPat], dtype = 'int64'), ignore_index = True)
            all_neg_patients = all_neg_patients[all_neg_patients != selectedPat]
            neg_patients = neg_patients.append(pd.Series([selectedPat2], dtype = 'int64'), ignore_index = True)
            all_neg_patients = all_neg_patients[all_neg_patients != selectedPat2]

            yearGap += 1
        #else :
         #   print('inf loop')

neg_patients.head()

0    1000100053877
1    6000600009289
2    4000000254832
3    4000000563007
4    4000000000423
dtype: int64

In [19]:
len(neg_patients.index)

3300

In [20]:
# TO DO
# 1. diseaseperiod
# 2. Redundant Visits
# 3. 3 < n < 51

In [21]:
# DiseaseCase
DiseaseCase['Disease'] = DiseaseCase['Disease'].replace("Diabetes Mellitus (ml)","Diabetes Mellitus") # Make all DM2 diagnosis' the same
DiseaseCase['DateOfOnset'] = pd.to_datetime(DiseaseCase.DateOfOnset,format='%Y-%m-%d')
DiseaseCase = DiseaseCase.sort_values(by='DateOfOnset', ascending = True)
DiseaseCase = DiseaseCase.drop_duplicates(subset = ['Patient_ID'])

In [22]:
neg_rec = lab.loc[lab['Patient_ID'].isin(neg_patients)]
neg_rec = pd.merge(neg_rec, DiseaseCase[['Patient_ID','DateOfOnset']], on='Patient_ID')

neg_rec.DateOfOnset = pd.to_datetime(neg_rec.DateOfOnset,format='%Y-%m-%d')

print(len(neg_rec.index)/ len(neg_patients))
neg_rec = neg_rec[neg_rec['PerformedDate'] > neg_rec['DateOfOnset']]
print(len(neg_rec.index)/ len(neg_patients))
print(neg_rec.dtypes)
neg_rec = neg_rec[(neg_rec['PerformedDate'] > neg_rec['DateOfOnset']) & (neg_rec['PerformedDate'] < neg_rec['DateOfOnset']+dp) ]
print(len(neg_rec.index)/ len(neg_patients))

77.95636363636363
60.60030303030303
Unnamed: 0                     int64
Patient_ID                     int64
PerformedDate         datetime64[ns]
Name_calc                     object
TestResult_calc              float64
UnitOfMeasure_calc            object
DateOfOnset           datetime64[ns]
dtype: object
43.67787878787879


In [23]:
# ## 7. Remove Redundent Visits later
neg_rec = neg_rec.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
neg_rec = neg_rec.reset_index(drop = True)

rem = []
for index, row in neg_rec.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))



749


In [24]:
## NEW
neg_rec_u = neg_rec
print(len(neg_rec_u.index))
neg_rec_u = neg_rec_u.drop(rem)
print(len(neg_rec_u.index))
print(len(neg_rec_u['Patient_ID'].unique()))

144137
143388
3048


In [25]:
neg_r_final = neg_rec_u[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() 
neg_r_final = neg_r_final[(neg_r_final['PerformedDate'] < 51) & (neg_r_final['PerformedDate'] > 3)]
neg_r_final

,PerformedDate
Patient_ID,
1000100000078,9
1000100000657,13
1000100005604,18
1000100008088,18
1000100009066,7
...,...
12120090033602,7
12120090039149,5
12120090042953,4


In [26]:
negFinal = neg_rec_u[neg_rec_u['Patient_ID'].isin(neg_r_final.index)]
negIDs = negFinal['Patient_ID'].unique()
negFinal.head()

,Unnamed: 0,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,2196754,1000100000078,2011-03-30,Alanine Aminotransferase in Serum or Plasma (ALT),9.00,U/L,2010-10-04
1,2196770,1000100000078,2011-03-30,Leukocytes in Blood (WBC),6.50,10^9/L,2010-10-04
2,2196771,1000100000078,2011-03-30,Erythrocytes in Blood (RBC),3.43,10^12/L,2010-10-04
3,2196764,1000100000078,2011-03-30,Total Bilirubin (TBIL),11.00,umol/L,2010-10-04
4,2196772,1000100000078,2011-03-30,Hemoglobin (Hb),117.00,g/L,2010-10-04


In [27]:
Results = pd.concat([posFinal, negFinal])

In [ ]:
names = pd.Series(tests)
turtile1 = []
turtile2 = []
for test in tests:
    turtile1.append(lab.loc[lab.Name_calc == test].TestResult_calc.quantile(0.3333))
    turtile2.append(lab.loc[lab.Name_calc == test].TestResult_calc.quantile(0.6667))
t1 = pd.Series(turtile1)
t2 = pd.Series(turtile2)
turtiles = pd.DataFrame(data = {'TestName' : names, 't1' : t1, 't2' : t2})

In [ ]:
for index, row in Results.iterrows() :
    test = row['Name_calc']
    value = row['TestResult_calc']
    #feature = [0,0,0]
    t1 = float(turtiles.loc[turtiles['TestName'] == test]['t1'])
    t2 = float(turtiles.loc[turtiles['TestName'] == test]['t2'])
    #print(t1,t2,value)
    if value <= t1 :
        feature = 1.0 #[1,0,0]
    elif value >= t2 :
        feature = 3.0 #[0,0,1]
    else :
        feature = 2.0 #[0,1,0]
    Results.at[index, 'TestResult_calc'] = feature
Results.head()      

In [ ]:
Results.groupby('TestResult_calc').agg('count')

In [ ]:
Results = Results.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
Results = Results.reset_index(drop = True)
rem = []
for index, row in Results.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))


In [ ]:
Results_u = Results
print(len(Results_u.index))
Results_u = Results_u.drop(rem)
print(len(Results_u.index))

In [ ]:
AllVisits = {}
VisitValues = {}
for index, row in Results_u.iterrows() :
    temp = (str(row['Patient_ID']),str(row['PerformedDate']))
    diag = (row['Name_calc'],row['TestResult_calc'])
    if temp not in AllVisits.keys():
        AllVisits[temp] = []
    AllVisits[temp] = AllVisits[temp] + [diag]

PVDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AllVisits.items() ]))
PVDF = PVDF.transpose()
PVDF = PVDF.dropna(axis=0,how='all') # Remove NA


In [ ]:
eTests = []
for t in tests :
    for i in range(1,4) :
        eTests.append(t+str(i))

In [ ]:
RowFeatures = pd.DataFrame(data = 0,
                       index = PVDF.index,
                       columns=eTests)
RowFeatures

In [ ]:
for index, row in RowFeatures.iterrows() :
    temp = PVDF.loc[index].dropna()
    for t in temp :
        test = t[0] + str(int(t[1]))
        row[test] = 1
RowFeatures

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 18)
visits_emb = svd.fit_transform(RowFeatures)
reducedDF = pd.DataFrame(data=visits_emb, index=RowFeatures.index)

In [ ]:
IDs = negIDs

In [ ]:
VisitHistorySVD = {}
for index, row in reducedDF.iterrows() :
    temp = int(index[0])
    featureVector = list(row)
    if temp not in VisitHistorySVD.keys():
        if temp in IDs :
            VisitHistorySVD[temp] = [0]
        else :
            VisitHistorySVD[temp] = [1]
    VisitHistorySVD[temp] = VisitHistorySVD[temp] + featureVector




In [ ]:
m = [0]
for k,v in VisitHistorySVD.items() :
    temp = len(v)
    if temp > m[-1] :
        m.append(k)
        m.append(temp)
        
print(m)

In [ ]:
# Fill in to 900
for k,v in VisitHistorySVD.items() :
    fillLen = 901 - len(v)
    VisitHistorySVD[k] = v + [0]*fillLen

In [ ]:
# MODELING
# Convert data to x y
x, y = [], []
for k,v in VisitHistorySVD.items() :
    if v[0] == 1 :
        y.append([1,0])
    if v[0] == 0 :
        y.append([0,1])
    x.append(v[1:])

print(len(x), len(y))

In [ ]:
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding,GRU, LSTM, GlobalMaxPooling1D, SpatialDropout1D, GRU, Bidirectional, Input, TimeDistributed, Reshape

In [ ]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, shuffle = True)
print(len(x_train), 'training')
print(len(x_test), 'testing')
#print(x_test[:10], y_test[:10])

In [ ]:
#x_test[0]

In [ ]:
# Bidirectional GRU

# Random search drop out rate and number of units/neurons
x_train = np.array(x_train)
x_test = np.array(x_test)
input_shape = (x_train.shape[1], 1) #900,1

units = 100


model = Sequential()
model.add(Reshape((50, 18), input_shape=(900,)))
# 1 sees the past 
# 2 sees the past and the future


dropout = 0.2 # for regularization
print('Number of hidden units: ', units, 'Dropout: ', dropout)
model.add(Bidirectional(GRU(units, input_shape=input_shape))) # 64, $128$, 256, 512
model.add(Dropout(dropout))
model.add(Dense(2, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print('Training...')
history = model.fit(
    np.array(x_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 25,
    validation_split = 0.10
)




In [ ]:
print(model.evaluate(x_test, np.array(y_test)))
preds = model.predict(x_test)

In [ ]:
#model.save(savename)

In [ ]:
from matplotlib import pyplot as plt
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

In [ ]:
# 70s val accuracy, 50-low60s test accuracy LSTM BOOHOO
# 67 val accuracy, 73% accuracy, auc_roc 0.836 GRU
# 0.7426 val accuracy, 0.7815 accuracy, roc 0.8745557924662403 GRU

In [ ]:
# TODOS : 
# 1. Check features
# 2. Try without SVD
# 3. Try with different diseases
# 4. 